<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/market.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Market クラス解説

Marketクラスは、取引所で扱われる通貨ペアーの１組合わせをモデリングしたクラスです。
現在はBinanceの USDBUSDのSPOT取引しか扱えませんが、比較的容易に拡張できるようにつくっていますので順次拡張していきます。

## Marketクラスメソッド

### マーケットオブジェクトの生成(`open`)

`open`メソッドでMaketオブジェクトを開きます。内部的にキャッシュされているので２回目からはキャッシュを返します。そのため複数回呼び出してOKです。

今後SPOT以外にも対応した場合、パラメータが変更になる可能性があります。
```
binance = Market.open(
    'BN',               # Binanceは 'BN'
    'BTCBUSD'
)
```

### データのダウンロード(`download`)

指定した日付あらあとのデータをダウンロードしてDBへ保存します。
BTCBUSDしか対応していないため効果はありませんが、`open`で開いたマーケットオブジェクト全てに対してダウンロード処理をします。

```
Market.download(
    5   #ダウンロード日数（N日前を指定)
    )
```


## Marketオブジェクトメソッド

Marketインスタンスが`binance`オブジェクトに保存されている前提で説明します。

###　情報表示　

####　`_repr_html_()`

jupyuter note上でオブジェクトを入力してリターンすると内部的に`_repr_html_()`がよばれて内容が分かりやすく表示されます。
```
binance
```

#### DBのパス表示 `file_path`

DBは巨大になりますので、不要になったら`file_path`プロパティで表示される場所を確認して手動で消してください。

```
binance.file_name
```

### OHLCV取得（pd.DataFrame形式)

時刻はμsで指定する。開始時刻に0を指定するとDBの最初から、終了に0を指定するとDBの最後まで（現在時刻に一番ちかいときまで）を指定したことになる。

```
binance.ohlcv(
    start_timestamp_us,     #開始時刻（μs）
    end_timestamp_us,       #終了時刻（μs)
    60                      #足の幅（秒）
    )
```

pd.DataFrame 形式でOHLCVデータが取得できます。この機能だけつかっても便利に使えます。
なお、Tickデータから任意の足を再生成しているので初回キャッシュされていないときはかなり時間がかかります。




## 以下動かしながら説明

### Google Collab向けパッケージインストール

In [ ]:
# Google Clab用です。必要に応じてコメントアウトしてください
! pip install --upgrade pip
! pip install numpy
! pip install pandas
! pip uninstall -y panel
! pip install --upgrade bokeh >= 3

# https://github.com/yasstake/rusty-bot/releases/
# （環境にあわせて修正お願いします。404エラーが出た場合は新しいリリースになっているので上記URLを確認してください）
# Linux/Google Collab用
!  pip install https://github.com/yasstake/rusty-bot/releases/download/release-0.2.0a/rbot-0.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


### 以後Marktライブラリの動作

In [39]:
# 必要ライブラリのimport
import rbot
from rbot import Market
from rbot import DAYS_BEFORE

In [40]:
# Binance('BN')の'BTCBUSD'ペアーの取引マーケットを生成
binance = Market.open('BN', 'BTCBUSD')

# 上記でオープンしたマーケットに対し5日より新しいログをダウンロード
Market.download(5)

# Market.download(5, True)       #ディフォルトではキャッシュされているので、再ダウンロードしたい場合は強制フラグをTrueにする。Collabでは１日あたり1分ぐらいかかります。

Start download from 2023-08-02T05:42:10.568277
thread endopen database /Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/SPOT-BTCBUSD.db
journal_mode = wal
wal mode already set
set as wal mode


In [41]:
# Marketオブジェクトが管理するDBに保存されているデータ期間を表示する。
binance

start,end
1690848007985000,1691366391123000
"""2023-08-01T00:00:07.985000""","""2023-08-06T23:59:51.123000"""
days=,5


In [42]:
# データベースのDisk上の場所を表示する。不要になったら手で消してください。

binance.file_name

'/Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/SPOT-BTCBUSD.db'

## OHLCVの取得

In [43]:
# 初回は遅いです。２回目はキャッシュされているので比較的高速です
binance.ohlcv(
    DAYS_BEFORE(5),     # 5日前から
    0,                  # 最新まで
    60*60
)

time_stamp,open,high,low,close,vol,count
i64,f64,f64,f64,f64,f64,u32
1691038800000000,29116.0,29140.54,29116.0,29136.01,7.16407,308
1691042400000000,29136.0,29143.84,29033.86,29076.0,69.45904,2034
1691046000000000,29074.0,29091.26,29009.09,29032.0,108.45396,2981
1691049600000000,29031.99,29099.99,28953.16,29097.91,152.76106,4035
1691053200000000,29099.08,29141.43,29084.68,29134.22,66.1349,1961
1691056800000000,29134.23,29176.67,29134.0,29154.0,36.44982,1524
1691060400000000,29154.0,29168.0,29110.0,29124.5,43.27512,1402
1691064000000000,29124.5,29140.0,29056.87,29084.0,66.71016,1933
1691067600000000,29085.99,29206.0,29085.99,29167.24,144.81476,2832


短めの足では任意のサイズの足が作れるので、このOHCLVだけを利用して独自の分析を行うにも便利だと思います。
週足より長い足も作れますが、性能が出ないのとデータが揃わない点に問題があり実用できではないかもしれません。

ご活用ください。

# おまけ

## pandasで作ったDBを読み込む

以下の方法でpythonのプログラムへSQLDBから直接データをロードできる。

In [44]:
import sqlite3
import pandas as pd

file = binance.file_name

connection = sqlite3.connect(file)

df = pd.read_sql_query('select * from trades order by time_stamp limit 10000000', connection)
df['timestamp'] = pd.to_datetime(df['time_stamp'], unit='us')
df = df.set_index('timestamp', drop=True)

df = df.drop(columns='time_stamp')
df

,action,price,size,id
timestamp,,,,
2023-08-01 00:00:07.985,Buy,29234.02,0.02500,989225825
2023-08-01 00:00:07.985,Buy,29234.00,0.00340,989225826
2023-08-01 00:00:07.985,Buy,29233.89,0.01799,989225827
2023-08-01 00:00:07.985,Buy,29233.29,0.00751,989225828
2023-08-01 00:00:07.985,Buy,29233.25,0.01938,989225829
...,...,...,...,...
2023-08-06 23:59:12.987,Buy,29054.47,0.02797,989589695
2023-08-06 23:59:12.987,Buy,29054.47,0.00706,989589696
2023-08-06 23:59:18.461,Buy,29054.47,0.01721,989589697


## ohlcを作る

`resample`を使えば任意の足のohlcvが作れる。
dfに入っているデータの最後は時間きっちりではなくて、件数指定でたまたま入ってきた切れ目なので未確定足になる。

In [45]:
ohlcv = df['price'].resample('60S').ohlc()
ohlcv['volume'] = df['size'].resample('60S').sum()
ohlcv


,open,high,low,close,volume
timestamp,,,,,
2023-08-01 00:00:00,29234.02,29234.02,29230.05,29230.05,0.55579
2023-08-01 00:01:00,29230.01,29230.01,29230.00,29230.01,0.21606
2023-08-01 00:02:00,29230.00,29230.01,29228.78,29228.78,0.16783
2023-08-01 00:03:00,29228.77,29230.00,29228.77,29229.99,0.08343
2023-08-01 00:04:00,29230.00,29232.00,29230.00,29231.99,0.66125
...,...,...,...,...,...
2023-08-06 23:55:00,29054.00,29054.00,29053.18,29053.19,0.29156
2023-08-06 23:56:00,29053.19,29056.00,29053.19,29056.00,0.04858
2023-08-06 23:57:00,29055.99,29055.99,29055.99,29055.99,0.06977


## rbot.Market.ohlcvとの比較

同じ処理をする`Market.ohlcv`と比較する。

In [46]:
# 開始時刻
start_time = ohlcv.index[0]
start_time

Timestamp('2023-08-01 00:00:00')

In [47]:
# 終了時刻
# ohldvの終了時刻は、最後の足の開始時刻のため１本前の足の終了時刻となる（この時間までで生成したohlcvは１本少なくなる）。
end_time = ohlcv.index[-1]
end_time

Timestamp('2023-08-06 23:59:00')

In [48]:
# pandasの時刻は'ns'なので1000分の１のusへ変換して表示
ohlcv2 = binance.ohlcv(int(start_time.value/1000), int(end_time.value/1000), 60*60)
ohlcv2

time_stamp,open,high,low,close,vol,count
i64,f64,f64,f64,f64,f64,u32
1690848000000000,29234.02,29289.99,29215.43,29286.0,35.80904,1557
1690851600000000,29286.01,29345.0,29177.0,29201.98,71.67005,2797
1690855200000000,29201.98,29218.0,28734.71,28935.59,578.81514,13158
1690858800000000,28935.58,28951.4,28818.08,28835.41,170.83839,3783
1690862400000000,28832.42,28924.65,28807.99,28868.0,129.98273,2831
1690866000000000,28867.99,28921.29,28854.0,28921.28,42.50483,1474
1690869600000000,28921.28,28967.74,28906.76,28915.99,47.04361,1982
1690873200000000,28914.78,28939.82,28898.43,28937.09,170.45635,3700
1690876800000000,28937.09,28986.3,28909.06,28986.3,83.1091,2484


想定どおり１本足が少なくなったがデータは一致している

## 性能比較

timeitを使って性能比較をする。

In [49]:
def panda_ohlcv():
    ohlcv = df['price'].resample('600S').ohlc()
    ohlcv['volume'] = df['size'].resample('60S').sum()

%timeit panda_ohlcv()

10.9 ms ± 656 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
def rust_ohlcv():
    ohlcv2 = binance.ohlcv(int(start_time.value/1000), int(end_time.value/1000), 60) 

%timeit rust_ohlcv()

7.51 ms ± 484 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Rust版は内部にキャッシュデータをもっていることもあって倍ぐらい早い傾向にある。